# 🤖 Template Base para Agents LangChain

**Objetivo:** Este notebook serve como um template completo e reutilizável para criar, testar e executar Agents com LangChain. Ele inclui as seções essenciais, desde a configuração até a execução, com exemplos práticos.

**Estrutura:**
1.  **Setup e Imports**: Bibliotecas essenciais.
2.  **Credenciais e Configurações**: API keys e configurações globais.
3.  **Modelos LLM**: Modelos de AI para diferentes tarefas.
4.  **Tools (Ferramentas)**: Um conjunto de ferramentas úteis e prontas para uso.
5.  **Prompts e Templates**: Estruturas de prompts para o agent.
6.  **Sistema RAG (Opcional)**: Lógica para busca em documentos.
7.  **Criação do Agent**: Montagem do agent com os componentes.
8.  **Execução e Processamento**: Função para invocar o agent.
9.  **Função Principal (Main)**: Orquestração e testes.

## 1. Setup e Imports

Nesta seção, importamos todas as bibliotecas necessárias para o funcionamento do nosso sistema. É uma boa prática manter todos os imports no início.

In [ ]:
# === IMPORTS BÁSICOS ===
import os
import asyncio
import time
from datetime import datetime
import json
import yaml

# === IMPORTS LANGCHAIN CORE ===
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.globals import set_llm_cache
from langchain_community.cache import InMemoryCache

# === IMPORTS PARA AGENTS ===
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import tool

# === IMPORTS PARA RAG ===
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

print("✅ Bibliotecas importadas com sucesso!")

## 2. Credenciais e Configurações

Aqui, configuramos as chaves de API e outras configurações globais. O método ideal é usar um arquivo `config.yaml` para manter as chaves seguras e fora do código.

In [ ]:
def setup_credenciais_e_config():
    """Carrega credenciais e define configurações globais."""
    try:
        # Carregar do config.yaml (método recomendado)
        with open("../config/config.yaml", "r") as config_file:
            config = yaml.safe_load(config_file)
        os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
        print("🔑 Credenciais carregadas do config.yaml")
        
    except FileNotFoundError:
        print("⚠️  Arquivo config.yaml não encontrado. Usando credenciais de ambiente (se existirem).")
        # Se o arquivo não existir, ele tentará usar variáveis de ambiente já configuradas
        pass

    # Configurar cache para otimização de custos e velocidade
    set_llm_cache(InMemoryCache())
    print("⚡ Cache em memória ativado.")

# Executar a configuração
setup_credenciais_e_config()

## 3. Modelos LLM

Definimos diferentes modelos de LLM para tarefas específicas. Isso permite otimizar o custo e a performance, usando modelos mais baratos para tarefas simples e modelos mais poderosos para análises complexas.

In [ ]:
def setup_modelos():
    """Configura e retorna um dicionário de modelos LLM."""
    modelos = {
        "rapido": ChatOpenAI(model='gpt-3.5-turbo', temperature=0.1, max_tokens=500),
        "premium": ChatOpenAI(model='gpt-4o', temperature=0.3, max_tokens=1500),
        "tecnico": ChatOpenAI(model='gpt-4', temperature=0, max_tokens=1000)
    }
    print(f"🤖 Modelos configurados: {list(modelos.keys())}")
    return modelos

# Inicializar modelos
modelos_llm = setup_modelos()

## 4. Tools (Ferramentas)

Esta é a seção mais importante para a funcionalidade do Agent. Aqui, definimos as ferramentas que o Agent pode usar para interagir com o mundo exterior, buscar informações ou executar ações.

In [ ]:
# === DEFINIÇÃO DE TOOLS ===

@tool
def buscar_informacao_web(termo: str) -> str:
    """Busca informações gerais na web sobre um termo específico. Use para perguntas abertas."""
    # Simulação de uma busca na web (pode ser integrado com DuckDuckGo, Google Search API, etc.)
    print(f"🔎 Buscando na web por: {termo}")
    return f"Dados simulados da web sobre '{termo}'. A web é vasta e cheia de informações." 

@tool
def calcular_expressao(expressao: str) -> str:
    """Calcula expressões matemáticas. Use para qualquer cálculo numérico."""
    try:
        # CUIDADO: eval() é inseguro em produção. Use uma biblioteca como 'numexpr' para segurança.
        resultado = eval(expressao, {"__builtins__": {}}, {})
        print(f"🧮 Calculando: {expressao} = {resultado}")
        return f"O resultado de '{expressao}' é {resultado}."
    except Exception as e:
        return f"Erro ao calcular '{expressao}': {e}"

@tool
def obter_data_e_hora_atual() -> str:
    """Retorna a data e a hora atuais. Use quando a pergunta envolver tempo."""
    now = datetime.now()
    data_hora = now.strftime("%d/%m/%Y, %H:%M:%S")
    print(f"🕒 Data e hora atual: {data_hora}")
    return f"A data e hora atuais são: {data_hora}."

@tool
def salvar_nota(conteudo: str) -> str:
    """Salva uma nota ou lembrete em um arquivo de texto. Use para persistir informações."""
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nome_arquivo = f"nota_{timestamp}.txt"
        with open(nome_arquivo, "w", encoding="utf-8") as f:
            f.write(conteudo)
        print(f"💾 Nota salva em: {nome_arquivo}")
        return f"Nota salva com sucesso no arquivo '{nome_arquivo}'."
    except Exception as e:
        return f"Erro ao salvar nota: {e}"

# === COLEÇÃO DE TOOLS ===
def criar_tools():
    """Retorna a lista de todas as tools disponíveis para o agent."""
    return [buscar_informacao_web, calcular_expressao, obter_data_e_hora_atual, salvar_nota]

lista_de_tools = criar_tools()
print(f"🛠️ {len(lista_de_tools)} ferramentas prontas para uso.")

## 5. Prompts e Templates

Definimos os templates de prompt que guiarão o comportamento do nosso Agent. O prompt do sistema é crucial para definir a "personalidade" e as instruções do Agent.

In [ ]:
def criar_prompts():
    """Cria e retorna os templates de prompt para o agent."""
    prompts = {
        "agent_system": ChatPromptTemplate.from_messages([
            ("system", """
            Você é um assistente de IA avançado e proativo.
            Seu objetivo é ajudar o usuário da forma mais completa possível.
            Você tem acesso a um conjunto de ferramentas para executar ações.
            
            Instruções:
            1. Analise a pergunta do usuário cuidadosamente.
            2. Se a pergunta pode ser respondida diretamente, responda.
            3. Se precisar de informações externas ou ações, escolha a ferramenta mais apropriada.
            4. Pense passo a passo sobre como usar as ferramentas para chegar à resposta final.
            5. Se uma ferramenta falhar, tente outra ou informe o usuário sobre a limitação.
            """),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}") # Onde o agent armazena seus pensamentos
        ])
    }
    print("📜 Prompts do agent criados.")
    return prompts

prompts_agent = criar_prompts()

## 6. Sistema RAG (Opcional)

Esta seção implementa um sistema de Retrieval-Augmented Generation (RAG). Ele permite que o Agent consulte uma base de conhecimento local (um arquivo de texto) para responder perguntas específicas, em vez de usar apenas seu conhecimento geral.

In [ ]:
def setup_rag(caminho_arquivo: str):
    """Configura um sistema RAG a partir de um arquivo de texto."""
    try:
        # Carregar documento
        loader = TextLoader(caminho_arquivo, encoding="utf-8")
        documents = loader.load()
        
        # Dividir em chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        
        # Criar embeddings e vector store
        embeddings = OpenAIEmbeddings()
        vectorstore = Chroma.from_documents(docs, embeddings)
        
        print(f"📚 Sistema RAG configurado com {len(docs)} chunks.")
        return vectorstore
    except Exception as e:
        print(f"⚠️  Erro ao configurar RAG: {e}. O RAG será desativado.")
        return None

# Exemplo de configuração do RAG (descomente para usar)
# print("--- Configurando RAG ---")
# vectorstore_rag = setup_rag("../data/base_conhecimento_britadeira.txt")

## 7. Criação do Agent

Aqui, juntamos todos os componentes: o modelo LLM, as ferramentas e o prompt para criar o Agent Executor, que é o cérebro que orquestra tudo.

In [ ]:
def criar_agent(modelo_llm, tools, prompt_system):
    """Cria e compila o Agent Executor."""
    agent = create_openai_functions_agent(
        llm=modelo_llm,
        tools=tools,
        prompt=prompt_system
    )
    
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,  # Mostra os "pensamentos" do agent
        return_intermediate_steps=False # Mude para True para debug detalhado
    )
    print("✅ Agent Executor criado com sucesso!")
    return agent_executor

# Criar o agent com os componentes definidos
agent_executor = criar_agent(modelos_llm["premium"], lista_de_tools, prompts_agent["agent_system"])

## 8. Execução e Processamento

Esta função encapsula a lógica de como uma pergunta do usuário é processada. Ela pode incluir a busca RAG antes de passar a pergunta para o Agent.

In [ ]:
async def processar_pergunta(pergunta: str, executor, vectorstore=None):
    """Processa uma pergunta, opcionalmente usando RAG, e invoca o agent."""
    print(f"\n--- 🚀 Processando Pergunta ---")
    print(f"🗣️ Usuário: {pergunta}")
    
    input_para_agent = {"input": pergunta}
    
    # Se o RAG estiver configurado, busca contexto e o adiciona ao input
    if vectorstore:
        docs_relevantes = vectorstore.similarity_search(pergunta, k=2)
        contexto_rag = "\n\n".join([doc.page_content for doc in docs_relevantes])
        input_para_agent["input"] += f"\n\n--- CONTEXTO ADICIONAL ---\n{contexto_rag}"
        print("📚 Contexto RAG adicionado.")
        
    # Invocar o agent
    resultado = await executor.ainvoke(input_para_agent)
    
    return resultado["output"]

print("⚙️ Função de processamento pronta.")

## 9. Função Principal (Main)

Este é o ponto de entrada para testar o sistema. Defina sua pergunta aqui e execute a célula para ver o Agent em ação.

In [ ]:
async def main():
    """Função principal para orquestrar e testar o agent."""
    
    # === PERGUNTA DE TESTE ===
    # Altere a pergunta abaixo para testar diferentes funcionalidades
    pergunta_teste = "Qual a data e hora de hoje? Depois, calcule (150 * 3) / 5 e salve o resultado em uma nota."
    
    # Para testar o RAG, primeiro configure o vectorstore na célula 6
    # e depois use uma pergunta relacionada ao documento.
    # pergunta_teste = "Como consertar uma britadeira que não liga?"
    
    # Processar a pergunta
    resposta_final = await processar_pergunta(
        pergunta=pergunta_teste, 
        executor=agent_executor, 
        vectorstore=None # Mude para vectorstore_rag para usar RAG
    )
    
    print("\n--- ✅ Resposta Final ---")
    print(resposta_final)

# === EXECUTAR O NOTEBOOK ===
# Usamos 'await' aqui porque o Jupyter/IPython já tem um loop de eventos rodando.
await main()